## Performance analysis of RL models

In [1]:
# Import required libraries for model analysis
import os
import pickle
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

# Example: Set the path to a model directory (change as needed)
model_dir = os.path.join('adapted_zhang_implementation', 'modelos_exitosos/n10_10amp_10prob')

# Paths to stats files
training_metrics_path = os.path.join(model_dir, 'training_metrics.pkl')
validation_metrics_path = os.path.join(model_dir, 'validation_metrics.pkl')

# Paths to model (e.g., best_model or final_model)
best_model_path = os.path.join(model_dir, 'best_model')
final_model_path = os.path.join(model_dir, 'final_model')

2025-09-02 17:28:58.607979: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-02 17:28:58.612261: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-02 17:28:58.665189: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-02 17:28:59.620215: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2025-09-02 17:28:59.620215: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/sofi/miniconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck

In [2]:
# Utility function to load training_results.txt as DataFrame
def load_training_results(path):
    import pandas as pd
    import os
    if os.path.exists(path):
        try:
            # Try common delimiters
            for sep in ['\t', ',', ';', ' ']:
                try:
                    df = pd.read_csv(path, sep=sep)
                    if df.shape[1] > 1:
                        return df
                except Exception:
                    continue
            # If all fail, try to read as a single column
            df = pd.read_csv(path, header=None)
            return df
        except Exception as e:
            print(f"Error loading {path}: {e}")
            return None
    else:
        print(f"File not found: {path}")
        return None

In [7]:
from ipywidgets import widgets
from IPython.display import display, clear_output

# Directory selection for modelos_exitosos
modelos_exitosos_dir = os.path.join('adapted_zhang_implementation', 'modelos_exitosos')

# List available subdirectories
available_dirs = [d for d in os.listdir(modelos_exitosos_dir) if os.path.isdir(os.path.join(modelos_exitosos_dir, d))]

model_dir_selector = widgets.Dropdown(
    options=available_dirs,
    description='Model Dir:',
    value=available_dirs[0] if available_dirs else None
)

# Output widget for dashboard
dashboard_output = widgets.Output()

def update_dashboard(change=None):
    with dashboard_output:
        clear_output(wait=True)
        selected_dir = model_dir_selector.value
        if selected_dir:
            model_dir = os.path.join(modelos_exitosos_dir, selected_dir)
            training_results_path = os.path.join(model_dir, 'training_results.txt')
            training_results_df = load_training_results(training_results_path)
            if training_results_df is not None:
                columns = list(training_results_df.columns)
                if len(columns) > 1:
                    x_selector = widgets.Dropdown(options=columns, description='X axis:')
                    y_default = 'max_fidelity' if 'max_fidelity' in columns else columns[0]
                    y_selector = widgets.Dropdown(options=columns, value=y_default, description='Y axis:')
                    bin_size_selector = widgets.IntSlider(value=100, min=1, max=1000, step=1, description='Bin size:')
                    output2 = widgets.Output()

                    def on_column_or_bin_change(change=None):
                        with output2:
                            clear_output(wait=True)
                            x = x_selector.value
                            y = y_selector.value
                            bin_size = bin_size_selector.value
                            x_data = training_results_df[x].values
                            y_data = training_results_df[y].values
                            if bin_size > 1:
                                n_bins = len(x_data) // bin_size
                                if n_bins > 0:
                                    x_binned = np.mean(x_data[:n_bins*bin_size].reshape(-1, bin_size), axis=1)
                                    y_binned = np.mean(y_data[:n_bins*bin_size].reshape(-1, bin_size), axis=1)
                                    plt.figure(figsize=(8,5))
                                    plt.plot(x_binned, y_binned, marker='o')
                                    plt.xlabel(f'{x} (binned, bin size={bin_size})')
                                    plt.ylabel(f'{y} (binned)')
                                    plt.title(f'{y} vs {x} (binned)')
                                else:
                                    plt.figure(figsize=(8,5))
                                    plt.plot(x_data, y_data, marker='o')
                                    plt.xlabel(x)
                                    plt.ylabel(y)
                                    plt.title(f'{y} vs {x}')
                            else:
                                plt.figure(figsize=(8,5))
                                plt.plot(x_data, y_data, marker='o')
                                plt.xlabel(x)
                                plt.ylabel(y)
                                plt.title(f'{y} vs {x}')
                            plt.grid(True)
                            plt.show()

                    x_selector.observe(on_column_or_bin_change, names='value')
                    y_selector.observe(on_column_or_bin_change, names='value')
                    bin_size_selector.observe(on_column_or_bin_change, names='value')
                    display(widgets.VBox([
                        widgets.Label(f'Select columns from {selected_dir}/training_results.txt to plot (with binning option):'),
                        widgets.HBox([x_selector, y_selector, bin_size_selector]),
                        output2
                    ]))
                    on_column_or_bin_change()
                else:
                    display(training_results_df)
            else:
                print('No training_results.txt data available in', selected_dir)
        else:
            print('No directory selected.')

In [8]:
# Directory selection for modelos_exitosos
modelos_exitosos_dir = os.path.join('adapted_zhang_implementation', 'modelos_exitosos')

# List available subdirectories
available_dirs = [d for d in os.listdir(modelos_exitosos_dir) if os.path.isdir(os.path.join(modelos_exitosos_dir, d))]

model_dir_selector = widgets.Dropdown(
    options=available_dirs,
    description='Model Dir:',
    value=available_dirs[0] if available_dirs else None
)

# Output widget for dashboard
dashboard_output = widgets.Output()

def update_dashboard(change=None):
    with dashboard_output:
        clear_output(wait=True)
        selected_dir = model_dir_selector.value
        if selected_dir:
            model_dir = os.path.join(modelos_exitosos_dir, selected_dir)
            training_results_path = os.path.join(model_dir, 'training_results.txt')
            training_results_df = load_training_results(training_results_path)
            if training_results_df is not None:
                columns = list(training_results_df.columns)
                if len(columns) > 1:
                    x_selector = widgets.Dropdown(options=columns, description='X axis:')
                    y_default = 'max_fidelity' if 'max_fidelity' in columns else columns[0]
                    y_selector = widgets.Dropdown(options=columns, value=y_default, description='Y axis:')
                    bin_size_selector = widgets.IntSlider(value=100, min=1, max=1000, step=1, description='Bin size:')
                    output2 = widgets.Output()

                    def on_column_or_bin_change(change=None):
                        with output2:
                            clear_output(wait=True)
                            x = x_selector.value
                            y = y_selector.value
                            bin_size = bin_size_selector.value
                            x_data = training_results_df[x].values
                            y_data = training_results_df[y].values
                            if bin_size > 1:
                                n_bins = len(x_data) // bin_size
                                x_binned = np.mean(x_data[:n_bins*bin_size].reshape(-1, bin_size), axis=1)
                                y_binned = np.mean(y_data[:n_bins*bin_size].reshape(-1, bin_size), axis=1)
                                plt.figure(figsize=(8,5))
                                plt.plot(x_binned, y_binned, marker='o')
                                plt.xlabel(f'{x} (binned, bin size={bin_size})')
                                plt.ylabel(f'{y} (binned)')
                                plt.title(f'{y} vs {x} (binned)')
                            else:
                                plt.figure(figsize=(8,5))
                                plt.plot(x_data, y_data, marker='o')
                                plt.xlabel(x)
                                plt.ylabel(y)
                                plt.title(f'{y} vs {x}')
                            plt.grid(True)
                            plt.show()

                    x_selector.observe(on_column_or_bin_change, names='value')
                    y_selector.observe(on_column_or_bin_change, names='value')
                    bin_size_selector.observe(on_column_or_bin_change, names='value')
                    display(widgets.VBox([
                        widgets.Label(f'Select columns from {selected_dir}/training_results.txt to plot (with binning option):'),
                        widgets.HBox([x_selector, y_selector, bin_size_selector]),
                        output2
                    ]))
                    on_column_or_bin_change()
                else:
                    display(training_results_df)
            else:
                print('No training_results.txt data available in', selected_dir)
        else:
            print('No directory selected.')

model_dir_selector.observe(update_dashboard, names='value')
display(widgets.VBox([widgets.Label('Select a model directory:'), model_dir_selector, dashboard_output]))
# Initial dashboard
update_dashboard()

In [11]:
# Load training and validation metrics from pickle files
def load_metrics(metrics_path):
    if os.path.exists(metrics_path):
        with open(metrics_path, 'rb') as f:
            metrics = pickle.load(f)
        return metrics
    else:
        print(f"File not found: {metrics_path}")
        return None

training_metrics = load_metrics(training_metrics_path)
validation_metrics = load_metrics(validation_metrics_path)

# Display keys to understand the structure
if training_metrics:
    print('Training metrics keys:', training_metrics.keys())
if validation_metrics:
    print('Validation metrics keys:', validation_metrics.keys())

# Plot training and validation statistics (example: loss and fidelity)
def plot_metrics(training_metrics, validation_metrics, metric_name):
    plt.figure(figsize=(8,5))
    if training_metrics and metric_name in training_metrics:
        plt.plot(training_metrics[metric_name], label=f'Training {metric_name}')
    if validation_metrics and metric_name in validation_metrics:
        plt.plot(validation_metrics[metric_name], label=f'Validation {metric_name}')
    plt.xlabel('Epoch')
    plt.ylabel(metric_name.capitalize())
    plt.title(f'{metric_name.capitalize()} over Epochs')
    plt.legend()
    plt.grid(True)
    plt.show()

# Example usage (uncomment and adjust metric names as needed):
# plot_metrics(training_metrics, validation_metrics, 'loss')
# plot_metrics(training_metrics, validation_metrics, 'fidelity')

Training metrics keys: dict_keys(['soft_success_training_rate', 'true_success_training_rate', 'training max fidelity', 'training average fidelity', 'average QValue', 'training_time', 'number_of_episodes', 'pruned'])
Validation metrics keys: dict_keys(['general_val_fidelity', 'average_val_fidelity', 'average_time_max_fidelity', 'average_QValue', 'validation_episodes'])
